# **Import **

In [ ]:
# =========== Import Packages ===========
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

!pip install jcopml

from jcopml.tuning.space import Real, Integer
from jcopml.utils import save_model, load_model

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation

from xgboost import XGBClassifier

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 102kB 3.8MB/s 
  Created wheel for jcopml: filename=jcopml-1.1.10-py2.py3-none-any.whl size=33232 sha256=cb739d70c89201e955fe7744c2deb46da780f1a058b503bd424e7108f79bab36
  Stored in directory: /root/.cache/pip/wheels/82/22/ce/b8b4c3400f788e5050b5b4b4d988884b8ec72b6b8751693914
Successfully built jcopml


In [ ]:
from google.colab import drive
 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:

# =========== Variabel for stopwords + punctuation's filtering ===========
sw_indo = stopwords.words('indonesian') + list(punctuation)

# =========== Import Dataset ===========
#pd.set_option("display.max_colwidth", 150)
df = pd.read_excel('/content/gdrive/My Drive/BDC SD/Folder Data Abis Preprocessing Baru/datatrainbr.xlsx')
df = df.drop('Unnamed: 0', axis=1)
df.head(10)

,ID,label,judul,narasi,total
0,71,1,Pemakaian Masker Menyebabkan Penyakit Legionnaires,A caller to a radio talk show recently shared that his wife was hospitalized n told she had COVID n only a couple of days left to live . A doctor ...,pemakaian masker menyebabkan penyakit legionnaires a caller to a radio talk show recently shared that his wife was hospitalized told she had only...
1,461,1,Instruksi Gubernur Jateng tentang penilangan bagi yg tidak bermasker di muka umum Rp.150.000 menggunakan e-tilang Via apps PIKOBAR,Yth.Seluruh Anggota Grup Sesuai Instruksi Gubernur Jawa Tengah Hasil Rapat Tim Gugus Tugas Covid 19 Jateng sbb:,instruksi gubernur jateng penilangan bermasker muka via apps anggota grup sesuai instruksi gubernur jawa hasil rapat tim gugus tugas 19 jateng sbb
2,495,1,Foto Jim Rohn: Jokowi adalah presiden terbaik dlm sejarah bangsa Indonesia,"Jokowi adalah presiden terbaik dlm sejarah bangsa Indonesia. Jim Rohn, motivator terbaik dunia",foto jim jokowi presiden terbaik sejarah bangsa indonesia jokowi presiden terbaik sejarah bangsa indonesia jim motivator terbaik dunia
3,550,1,"ini bukan politik, tapi kenyataan Pak Jokowi berhasil memulangkan 11,000 Triliun uang negara dari Swiss.","Maaf Mas2 dan Mbak2, ini bukan politik, tapi kenyataan Pak Jokowi berhasil memulangkan 11,000 Triliun uang negara dari Swiss. 11ribu Triliun siap ...",politik kenyataan jokowi berhasil memulangkan 11000 triliun uang negara swiss maaf mas2 mbak2 politik kenyataan jokowi berhasil memulangkan 11000...
4,681,1,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,foto lihat foto panas dingin lihat foto panas dingin
5,736,1,event promo smartphone JNE 2020 spesial di bulan juni,"selamat siang teman teman fb ku semuanya🤩,cuman mau memberi informasi dan rezeky gak kemana ya😊, kemarin saya mengikuti event promo smartphone JNE...",event promo smartphone jne 2020 spesial juni selamat siang teman teman fb ku informasi rezeky kemana kemarin mengikuti event promo smartphone jne...
6,1161,1,MEREKA SUDAH MEMPERSIAPKAN DIRI DENGAN BAIK UNTUK MENGUASAI NEGRI INI!,LIHATLAH MEREKA SUDAH MEMPERSIAPKAN DIRI DENGAN BAIK UNTUK MENGUASAI NEGRI INI!,menguasai negri lihatlah menguasai negri
7,1597,1,Foto Presiden Italia menangis karena tak cukup lahan untuk mengubur 700an korban corona per hari,Italia punya fasilitas perawatan kesehatan tercanggih tapi semuanya gagal mengendalikan corona karena awalnya itu hanya dianggap sekadar lelucon d...,foto presiden italia menangis lahan mengubur 700an korban corona italia fasilitas perawatan kesehatan tercanggih gagal mengendalikan corona diang...
8,2098,1,"Kapolres Timor Tengah Utara , Nusa Tenggara Timur Beri Klarifikasi Terkait Video Polisi Diamkan Korban Kecelakaan",Polisi goblok. Bukanya cpet” d tolongin malah d liati aja,kapolres timor utara nusa tenggara timur klarifikasi terkait video polisi diamkan korban kecelakaan polisi goblok bukanya cpet tolongin
9,2226,1,Video Polisi china telah menganiaya wanita uighur lehernya di cekik pake kaki,"Polisi china telah menganiaya wanita uighur lehernya di cekik pake kaki ,melek tu yg bilang tidak ada penganiayaan Dan Ham diam seribu bahasa … Se...",video polisi china menganiaya wanita uighur lehernya cekik pake kaki polisi china menganiaya wanita uighur lehernya cekik pake kaki melek tu peng...


In [ ]:
# =========== Data Splitting ===========
x_train = df.total
y_train = df.label

In [ ]:
df_t = pd.read_excel('/content/gdrive/My Drive/BDC SD/Folder Data Abis Preprocessing Baru/datatestbr.xlsx')
df_t = df_t.drop('Unnamed: 0', axis=1)
df_t.label = df_t.label.fillna(0)
df_t.label = df_t.label.astype(int)
df_t.head(10)
#df_t.info()

,ID,judul,narasi,label,total
0,238057,Narasi Tito Karnavian Berideologi Komunis Karena Pernah Disekolahkan Partai Komunis China di Beijing,TITO KARNIVAN ITU BERIDIOLOGI KOMUNIS DIA BISA DI KATAKAN PKI KARENA DI PERNAH DI SEKOLAHLAH OLEH PARTAI KOMUNIS CHINA DI BAIJING,1,narasi tito berideologi komunis disekolahkan partai komunis china beijing tito beridiologi komunis pki sekolahlah partai komunis china
1,238158,Anies: Seberat beratnya Pekerjaan Akan terasa ringan Bila tidak di kerjakan,Seberat beratnya Pekerjaan Akan terasa ringan Bila tidak di kerjakan,1,anies seberat beratnya pekerjaan ringan kerjakan seberat beratnya pekerjaan ringan kerjakan
2,238865,Hindu di india Melemparkan Patung Buatan Mereka Ke Laut Karena Tidak Bisa Menolong Mereka Dari Corona,Hindu di india melemparkan patung buatan mereka ke laut karena apa yang mereka lakukan tidak dapat membela mereka dari dari virus corona Maha Suci...,1,hindu india melemparkan patung buatan laut menolong corona hindu india melemparkan patung buatan laut lakukan membela virus corona maha suci alla...
3,248298,RSCM Praktekkan Penyedotan Plug Vena/Saluran Darah,"Mulai Hari ini di RSCM mulai diPraktekkan Penyedotan Plug (Sumbatan) di Vena/Saluran Darah Ke Jantung, jadi Tidak Pake Ring atau Bypass lagi.\n \n...",1,rscm praktekkan penyedotan plug darah rscm dipraktekkan penyedotan plug sumbatan darah jantung pake ring bypass semoga teknologi membawa khabar g...
4,255176,Permohonan Kelonggaran Angsuran ke OJK,"Untuk sekedar info, Bagi anda yg punya ansuran/cicilan namun belum dapat kelonggaran dari leasing/pembiayaan silahkan buat pengajuan ke kantor OJK...",1,permohonan kelonggaran angsuran ojk info kelonggaran silahkan pengajuan kantor ojk otoritas jasa keuangan proses online pengajuan padat silahkan ...
5,267051,Aksi BEM se-Jabotabek hari ini,"Assalamua’laikum warahmatullahi wabarakatuh.\n \n Jangan lihat TV ndak ada ini…!!!\n Lihat Sosmed aja,kita viralkan…!!!",1,aksi bem assalamualaikum warahmatullahi wabarakatuh lihat tv ndak lihat sosmed viralkan
6,269336,Sinar Kosmik Berbahaya Radiasi,"Malam ini 00:30-03:30 pastikan untuk mematikan telepon: TV Singapura telah mengumumkan berita. Tolong selepas baca ini, lindungi diri anda. Berita...",1,sinar kosmik berbahaya radiasi malam pastikan mematikan telepon tv singapura mengumumkan berita tolong selepas baca lindungi beritahukan kerabat ...
7,272438,Pihak SMKN 12 Beri Klarifikasi Perihal Kabar Viral Tak Pasang Foto Presiden Joko Widodo,Lokasi di SMKN 12 Jakarta Utara. Sekolah Negeri tapi tidak memasang foto Presiden…!!! 😡\n \n Mohon viralkan…!!!\n \n Sampai mereka memasang Foto P...,1,smkn 12 klarifikasi perihal kabar viral pasang foto presiden joko widodo lokasi smkn 12 jakarta utara sekolah negeri memasang foto presiden mohon...
8,282152,Pembuatan KTP Orang Gila Demi Ambisi Kekuasaan,PARAH…MEMAKSAKAN HAL YANG TAK WAJAR TERUS DILAKUKAN DEMI AMBISI KEKUASAAN… Pembuatan KTP buat orang2 gila..Beneran ada di bekasi.,1,pembuatan ktp orang gila ambisi kekuasaan wajar ambisi kekuasaan pembuatan ktp orang2 bekasi
9,288490,Daniel Radcliffe Positiv Terinfeksi Corona Covid-19,BREAKING: Daniel Radcliffe test posotive for coronavirus. The actor is said to be the first famous person to publicly confirmed.,1,daniel positiv terinfeksi corona breaking daniel test posotive for coronavirus the actor is said to be the first famous person to publicly confirmed


In [ ]:
df_t.label.value_counts()

1    395
0     75
Name: label, dtype: int64

In [ ]:
x_test = df_t.total
y_test = df_t.label

# **Modelling**

In [ ]:
# =========== Use Pipeline ===========
pipeline = Pipeline([
    ('prep', TfidfVectorizer(tokenizer=word_tokenize, stop_words=sw_indo)),
    ('algo', XGBClassifier(n_jobs=-1, random_state=42))
])

In [ ]:
# =========== XGBoost's Parameter for the case ===========
parameter = {
 'algo__max_depth': Integer(low=12, high=20),
 'algo__learning_rate': Real(low=-2, high=0, prior='log-uniform'),
 'algo__n_estimators': Integer(low=195, high=210),
 'algo__subsample': Real(low=0.5, high=1, prior='uniform'),
 'algo__gamma': Integer(low=1, high=10),
 'algo__colsample_bytree': Real(low=0.1, high=1, prior='uniform')
}

In [ ]:

model = RandomizedSearchCV(pipeline, parameter, cv=5, n_iter=50, n_jobs=-1, verbose=1, random_state=50)
model.fit(x_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 40.1min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 49.1min finished
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('prep',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                                        

In [ ]:
# =========== Model Evalutation ===========
print(model.best_params_)

print(model.score(x_train, y_train), model.score(x_test, y_test))

pred = model.predict(x_test)
#print(classification_report(y_test, pred))

from sklearn.metrics import f1_score
print('Accuracy: ', f1_score(y_test, pred))
print()

{'algo__colsample_bytree': 0.8230451777895204, 'algo__gamma': 8, 'algo__learning_rate': 0.3940217676386827, 'algo__max_depth': 17, 'algo__n_estimators': 203, 'algo__subsample': 0.9268612478927063}
0.8709524935003545 0.9106382978723404
Accuracy:  0.948905109489051



In [ ]:
# =========== Confusion Matrix Visualization ===========

print('Confusion Matrix: \n', confusion_matrix(y_test, pred))

Confusion Matrix: 
 [[ 38  37]
 [  5 390]]


In [ ]:
from google.colab import files

df_p = pd.DataFrame(df_t.ID)
df_p['label'] = pred
df_p.head()

,ID,label
0,238057,1
1,238158,1
2,238865,1
3,248298,1
4,255176,1


In [ ]:
df_p.label.value_counts()

1    427
0     43
Name: label, dtype: int64

In [ ]:
from google.colab import files
df_p.to_excel('prediksixgb.xlsx') 
files.download('prediksixgb.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>